In [2]:
import os 
import pandas as pd 
import numpy as np
import json 
import time 
import random 
import math 
import re 
import requests 
import datetime
import sqlite3
from functools import partial 
from collections import deque, defaultdict
from contextlib import contextmanager
from datetime import date, datetime as dt, timezone

import psycopg2
import psycopg2.extras
import altair as alt 
from bs4 import BeautifulSoup
from pprint import PrettyPrinter
from web3 import Web3
from etherscan import Etherscan
from dotenv import load_dotenv 
from attrdict import AttrDict
from pycoingecko import CoinGeckoAPI
from IPython.display import display, HTML

import warnings
warnings.filterwarnings('ignore')

load_dotenv()

pprint = PrettyPrinter(indent=4)

In [3]:
API_KEY = os.environ["ETHERSCAN_API_KEY"]
addr = os.environ["MM_HOT_ADDRESS"]
w3 = Web3(Web3.HTTPProvider(f'https://mainnet.infura.io/v3/{os.environ["INFURA_PROJECT_ID"]}'))
eth = Etherscan(API_KEY, net="MAIN") 
cg = CoinGeckoAPI()

In [4]:
UNISWAP_V2_ROUTER = "0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D"
OLYMPUS_DAO_STAKING_HELPER = "0xC8C436271f9A6F10a5B80c8b8eD7D0E8f37a612d"
OHM_TOKEN_CONTRACT = "0x383518188C0C6d7730D91b2c03a03C837814a899"
OHM_TREASURY_V2 = '0x31F8Cc382c9898b273eff4e0b7626a6987C846E8'
OHM_STAKING_V2 = "0xFd31c7d00Ca47653c6Ce64Af53c1571f9C36566a"
OHM_STAKING_V2_DEPLOYMENT_TX = "0x1a1a3dd33879ff1b765cc5fae84102990a3e866d1285e0d81d79cbff836f56e6"
OHM_STAKING_DISTRIBUTOR_V4 = '0xc58e923bf8a00e4361fe3f4275226a543d7d3ce6'

In [5]:
def date_to_datetime(d): 
    return dt(*d.timetuple()[:6], tzinfo=timezone.utc)

def utc_timestamp(dto): 
    if isinstance(dto, date): 
        return date_to_datetime(dto).replace(tzinfo=timezone.utc).timestamp()
    elif isinstance(dto, datetime): 
        return dto.replace(tzinfo=timezone.utc).timestamp()
    raise Exception("Expected input of either date or datetime")

def get_current_block_number(): 
    # get the most recent verified block number 
    now_plus_min = dt.now(tz=timezone.utc) + datetime.timedelta(minutes=1)
    timestamp = math.ceil(utc_timestamp(now_plus_min)) # seconds
    return int(eth.get_block_number_by_timestamp(timestamp, "before"))

def get_first_block_for_day(day: date): 
    # get the first block mined for a given day. handles case where first block is mined on exactly midnight 
    timestamp = int(utc_timestamp(date_to_datetime(day)))
    block = int(eth.get_block_number_by_timestamp(timestamp, "after"))
    block_data = eth.get_block_reward_by_block_number(block)
    true_datetime = dt.fromtimestamp(int(block_data["timeStamp"]), tz=timezone.utc)
    if true_datetime < date_to_datetime(day): 
        return block + 1
    else: 
        return block 
    
def get_last_block_for_day(day: date): 
    # get the last block mined for a given day
    timestamp = int(utc_timestamp((date_to_datetime(day) + datetime.timedelta(days=1))))
    block = int(eth.get_block_number_by_timestamp(timestamp, "before"))
    return block 

def get_block_range_for_day(day: dt.date): 
    return get_first_block_for_day(day), get_last_block_for_day(day)

def get_OHM_staking_block_range(): 
    """ Returns the full block range from when the staking contract was deployed until current moment in time """
    staking_deploy_tx = w3.eth.get_transaction(OHM_STAKING_V2_DEPLOYMENT_TX)
    staking_deploy_block = int(staking_deploy_tx["blockNumber"])
    cur_block = get_current_block_number()
    return staking_deploy_block, cur_block

d = dt.now().date() - datetime.timedelta(days=3)
bs = get_first_block_for_day(d)
be = get_last_block_for_day(d)
print(d)
print(bs, be)

2021-12-13
13793513 13799998


In [6]:
def get_connection(): 
    conn = psycopg2.connect("dbname=olympus user=postgres password=a")
    return conn 
    
@contextmanager
def db_conn():
    conn = get_connection()
    try:
        yield conn
    finally:
        conn.close()

@contextmanager
def db_cur(connection):
    cur = connection.cursor()
    try:
        yield cur
    finally:
        cur.close()

In [7]:
# https://etherscan.io/tokencheck-tool
OHM_TOKEN_CONTRACT_ADDRS = dict(
    sOHM='0x04f2694c8fcee23e8fd0dfea1d4f5bb8c352111f', 
    gOHM='0x0ab87046fBb341D058F17CBC4c1133F25a20a52f', 
    OHM='0x383518188c0c6d7730d91b2c03a03c837814a899', 
    wsOHM='0xca76543cf381ebbb277be79574059e32108e3e65'
)
float(eth.get_acc_balance_by_token_and_contract_address(OHM_TOKEN_CONTRACT_ADDRS['sOHM'], addr)) * 10**-9

6.671139212000001

In [8]:
def create_table_OHM_staking_internal_txs(conn, cur): 
    cur.execute("""
    CREATE TABLE IF NOT EXISTS OHM_stake_internal_txs (
        block_no int8 NOT NULL, 
        tx_hash bytea NOT NULL, 
        type VARCHAR(20) NOT NULL, 
        from_addr bytea NOT NULL, 
        to_addr bytea NOT NULL, 
        value numeric NOT NULL, 
        PRIMARY KEY (tx_hash, from_addr, to_addr, type, value)
    );
    """)
    conn.commit()
    
def insert_many_table_OHM_staking_internal_txs(conn, cur, df, kwargs=dict(template=None, page_size=100)): 
    records = [
        (
            r['block_no'], 
            r['txhash'], 
            r['type'], 
            r['from'], 
            r['to'], 
            0
        ) 
        for r in df.to_dict(orient="records")
    ]    
    insert_query = """
        INSERT INTO OHM_stake_internal_txs (
            block_no,
            tx_hash,
            type,
            from_addr,
            to_addr,
            value
        ) VALUES %s
        ON CONFLICT (tx_hash, from_addr, to_addr, type, value) DO NOTHING
    """
    psycopg2.extras.execute_values(cur, insert_query, records, **kwargs)
    conn.commit()   

def get_max_block_OHM_staking_internal_txs(conn, cur): 
    cur.execute("SELECT max(block_no) FROM OHM_stake_internal_txs")
    res = cur.fetchone()[0]
    return res if res else 0 

def get_rebase_txs_OHM_staking_internal_txs(conn, cur): 
    cur.execute("SELECT block_no, encode(tx_hash::bytea, 'escape') as txhash FROM OHM_stake_internal_txs")
    res = cur.fetchall()
    if res: 
        res = list(set(res))
        res.sort(key=lambda r: -r[0])
        return res 
    else: 
        return []

def staking_distributor_internal_txs_to_records(df): 
    records = []
    df['Block'] = df['Block'].fillna(-1).astype(int)
    for row in df.to_dict(orient="records"): 
        records.append({
            'block_no': row['Block'], 
            'txhash': row["Parent Txn Hash"], 
            'type': row['Type'], 
            'from': row['From'], 
            'to': row['To'], 
            'value': row['Value']
        })
    df = pd.DataFrame(records)
    tx_block = df.groupby(by=["txhash"])['block_no'].max()
    for txhash, block in zip(tx_block.index, tx_block.values): 
        df['block_no'].loc[df.txhash == txhash] = block
    df = df[df['block_no'] != -1]
    return df

def get_staking_distributor_internal_txs_df(max_block): 
    # fake a User-Agent as this bypasses captcha requirements 
    headers={'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
    url_base = f'https://etherscan.io/txsInternal?a={OHM_STAKING_DISTRIBUTOR_V4}&&m=advanced&p='
    i = 1
    dfs = []
    success = True 
    while success: 
        url = url_base + str(i)
        res = requests.get(url, headers=headers)
        soup = BeautifulSoup(res.content, 'html.parser')
        tables = [
            t for t in soup.find_all('table') 
            if "Txn Hash" in str(t) and re.sub("[\s\n\r]", "", t.find("tbody").getText())
        ]
        assert len(tables) == 1
        df = pd.read_html(tables[0].prettify())[0]
        print(f"Page: {i} - Record Count: {len(df)}")
        success &= len(df) > 1
        if success:
            df_new = staking_distributor_internal_txs_to_records(df)
            dfs.append(df_new)
            if df_new['block_no'].min() <= max_block: 
                break 
            i += 1
            time.sleep(3 * random.random() + 3 * random.random() + 3)
    return pd.concat(dfs)


In [9]:
with db_conn() as conn, db_cur(conn) as cur: 
    max_block = get_max_block_OHM_staking_internal_txs(conn, cur)
print(f"Max block in dataset: {max_block}")
df = get_staking_distributor_internal_txs_df(max_block)
print(len(df))

Max block in dataset: 13783000
Page: 1 - Record Count: 50
50


In [10]:
with db_conn() as conn, db_cur(conn) as cur: 
    create_table_OHM_staking_internal_txs(conn, cur)
    insert_many_table_OHM_staking_internal_txs(conn, cur, df)
    print(get_max_block_OHM_staking_internal_txs(conn, cur))

13783000


In [11]:
def create_table_OHM_rebase_txs(conn, cur): 
    cur.execute("""
    CREATE TABLE IF NOT EXISTS OHM_rebase_txs (
        block_no int8 NOT NULL, 
        tx_hash bytea NOT NULL, 
        block_time timestamptz NOT NULL, 
        rebase numeric NOT NULL, 
        epoch int8 NOT NULL, 
        PRIMARY KEY (tx_hash)
    );
    """)
    conn.commit()
    
def insert_many_table_OHM_rebase_txs(conn, cur, records, kwargs=dict(template=None, page_size=100)): 
    records = [
        (
            r.block_no, 
            r.tx_hash, 
            r.date, 
            r.rebase, 
            r.epoch
        ) 
        for r in records
    ]    
    insert_query = """
        INSERT INTO OHM_rebase_txs (
            block_no,
            tx_hash,
            block_time,
            rebase, 
            epoch
        ) VALUES %s
        ON CONFLICT (tx_hash) DO NOTHING
    """
    psycopg2.extras.execute_values(cur, insert_query, records, **kwargs)
    conn.commit()   

def get_tx_hashes_OHM_rebase_txs(conn, cur): 
    cur.execute("SELECT DISTINCT encode(tx_hash::bytea, 'escape') FROM OHM_rebase_txs")
    res = cur.fetchall()
    return [r[0] for r in res] if res else [] 
 
def get_all_table_OHM_rebase_txs(conn, cur, kwargs=dict(template=None, page_size=100)):   
    sql = """
        SELECT  block_no,
                encode(tx_hash::bytea, 'escape'),
                block_time AT TIME ZONE 'UTC',
                rebase, 
                epoch  
        FROM OHM_rebase_txs 
        ORDER BY epoch ASC 
    """
    cur.execute(sql)
    res = cur.fetchall()
    records = [
        AttrDict(
            block_no=r[0], 
            date=r[2].replace(tzinfo=timezone.utc), 
            rebase=float(r[3]), 
            epoch=r[4]
        ) 
        for r in res
    ]
    return records 
    
def parse_sohm_internal_tx_logs(txhash): 
    # txhash represents an on chain transaction that contains an internal transaction that interacted with the sOHM token contract 
    # we get the tx receipt for this tx, which contains logs of these interactions, then parse the interactions 
    # staking distributor etherscan internal txs: https://etherscan.io/address/0xc58e923bf8a00e4361fe3f4275226a543d7d3ce6/advanced#internaltx
    # https://docs.olympusdao.finance/main/basics/basics#how-do-i-track-my-rebase-rewards
    contract_sohm = w3.eth.contract(
        address='0x04F2694C8fcee23e8Fd0dfEA1d4f5Bb8c352111F', 
        abi='[{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"owner","type":"address"},{"indexed":true,"internalType":"address","name":"spender","type":"address"},{"indexed":false,"internalType":"uint256","name":"value","type":"uint256"}],"name":"Approval","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"uint256","name":"epoch","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"rebase","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"index","type":"uint256"}],"name":"LogRebase","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"stakingContract","type":"address"}],"name":"LogStakingContractUpdated","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"uint256","name":"epoch","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"timestamp","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"totalSupply","type":"uint256"}],"name":"LogSupply","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"previousOwner","type":"address"},{"indexed":true,"internalType":"address","name":"newOwner","type":"address"}],"name":"OwnershipPulled","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"previousOwner","type":"address"},{"indexed":true,"internalType":"address","name":"newOwner","type":"address"}],"name":"OwnershipPushed","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"from","type":"address"},{"indexed":true,"internalType":"address","name":"to","type":"address"},{"indexed":false,"internalType":"uint256","name":"value","type":"uint256"}],"name":"Transfer","type":"event"},{"inputs":[],"name":"DOMAIN_SEPARATOR","outputs":[{"internalType":"bytes32","name":"","type":"bytes32"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"INDEX","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"PERMIT_TYPEHASH","outputs":[{"internalType":"bytes32","name":"","type":"bytes32"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"owner_","type":"address"},{"internalType":"address","name":"spender","type":"address"}],"name":"allowance","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"value","type":"uint256"}],"name":"approve","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"gons","type":"uint256"}],"name":"balanceForGons","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"who","type":"address"}],"name":"balanceOf","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"circulatingSupply","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"decimals","outputs":[{"internalType":"uint8","name":"","type":"uint8"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"subtractedValue","type":"uint256"}],"name":"decreaseAllowance","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"gonsForBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"addedValue","type":"uint256"}],"name":"increaseAllowance","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"index","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"stakingContract_","type":"address"}],"name":"initialize","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"initializer","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"manager","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"name","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"owner","type":"address"}],"name":"nonces","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"owner","type":"address"},{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"},{"internalType":"uint256","name":"deadline","type":"uint256"},{"internalType":"uint8","name":"v","type":"uint8"},{"internalType":"bytes32","name":"r","type":"bytes32"},{"internalType":"bytes32","name":"s","type":"bytes32"}],"name":"permit","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"pullManagement","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"newOwner_","type":"address"}],"name":"pushManagement","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"profit_","type":"uint256"},{"internalType":"uint256","name":"epoch_","type":"uint256"}],"name":"rebase","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"","type":"uint256"}],"name":"rebases","outputs":[{"internalType":"uint256","name":"epoch","type":"uint256"},{"internalType":"uint256","name":"rebase","type":"uint256"},{"internalType":"uint256","name":"totalStakedBefore","type":"uint256"},{"internalType":"uint256","name":"totalStakedAfter","type":"uint256"},{"internalType":"uint256","name":"amountRebased","type":"uint256"},{"internalType":"uint256","name":"index","type":"uint256"},{"internalType":"uint256","name":"blockNumberOccured","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"renounceManagement","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"_INDEX","type":"uint256"}],"name":"setIndex","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"stakingContract","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"symbol","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"totalSupply","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"value","type":"uint256"}],"name":"transfer","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"from","type":"address"},{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"value","type":"uint256"}],"name":"transferFrom","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"}]'
    )
    receipt = w3.eth.get_transaction_receipt(txhash)
    if receipt.status == 0: 
        print("Skipping failed transaction")
        return None 
    logs_rebase = contract_sohm.events.LogRebase().processReceipt(receipt)
    logs_supply = contract_sohm.events.LogSupply().processReceipt(receipt)
    if not len(logs_rebase) == len(logs_supply) == 1: 
        print("Skipping transaction that doesn't have logSupply and logRebase events")
        return None 
    logs_rebase = logs_rebase[0]
    logs_supply = logs_supply[0]
    assert logs_rebase.args.epoch == logs_supply.args.epoch
    epoch = logs_rebase.args.epoch
    rebase = logs_rebase.args.rebase
    index = logs_rebase.args.index 
    totalSupply = logs_supply.args.totalSupply
    return AttrDict(
        tx_hash=txhash, 
        epoch=epoch, 
        rebase=rebase, 
        index=index, 
        totalSupply=totalSupply
    )  

def _log_rebase(r, b): 
        rstr = f"{(r['rebase'] / 100):.7f}"
        print(f"{r.date} block: {r.block_no} epoch: {r.epoch} Rebase: ({rstr}): {b} -> {b * (1 + r.rebase)}")

def _log_transfer(t, b):
    print(f"{t.date} block: {t.block_no} Transfer: {b} -> {b + t.value}")

def get_ohm_token_balance_for_address(addr, dmin, dmax, rebases, bs=0, be=99999999, sort='asc', log=True): 
    ohm_tokens = ["OHM", "sOHM", "gOHM", "wsOHM"]
    vmul = dict(OHM=1e-9, sOHM=1e-9, gOHM=1e-18, wsOHM=1e-18)
    res = eth.get_erc20_token_transfer_events_by_address(addr, bs, be, sort)
    # create a time series of transfer events for each OHM token 
    tsmap = defaultdict(list)
    balances = defaultdict(list)
    for tx in res: 
        symbol = tx['tokenSymbol']
        if symbol in ohm_tokens:
            d = dt.fromtimestamp(int(tx["timeStamp"]), tz=timezone.utc)
            sign = -1 if tx["from"].lower() == addr.lower() else 1 
            v = sign * float(tx['value']) * vmul[symbol]
            tsmap[symbol].append(AttrDict({
                "date": d, 
                "value": v, 
                "block_no": tx['blockNumber']
            }))
            if symbol != 'sOHM': 
                balances[symbol].append(AttrDict(
                    date=d, 
                    balance=v if not balances[symbol] else balances[symbol][-1].balance + v
                ))
    # Compute running balance for sOHM with rebases 
    symbol = 'sOHM'
    sohm_balance = 0
    sohm_balances = []
    sohm_transfers = deque(tsmap[symbol])
    rebases = deque(rebases) 
    log_rebase = _log_rebase if log else lambda *args: None
    log_transfer = _log_transfer if log else lambda *args: None
    while sohm_transfers or rebases:
        if sohm_transfers and not rebases: 
            # only transfers remain
            transfer = sohm_transfers.popleft()
            d = transfer.date
            log_transfer(transfer, sohm_balance)
            sohm_balance += transfer.value
        elif not sohm_transfers and rebases:
            # only rebases remain
            rebase = rebases.popleft() 
            d = rebase.date
            log_rebase(rebase, sohm_balance)
            sohm_balance *= (1 + rebase.rebase)
        else: 
            # both transfer and rebase events remain 
            if sohm_transfers[0].date <= rebases[0].date: 
                # transfer occurred first 
                transfer = sohm_transfers.popleft()
                d = transfer.date
                log_transfer(transfer, sohm_balance)
                sohm_balance += transfer.value
            else: 
                # rebase occurred first
                rebase = rebases.popleft() 
                d = rebase.date
                log_rebase(rebase, sohm_balance)
                sohm_balance *= (1 + rebase.rebase)
        if balances[symbol] or sohm_balance != 0: 
            balances[symbol].append(AttrDict(
                date=d, 
                balance=sohm_balance, 
            ))
    return balances, tsmap


In [12]:
with db_conn() as conn, db_cur(conn) as cur: 
    create_table_OHM_rebase_txs(conn, cur)
    # get existing internal txs that represent interactions with sOHM token contract 
    txs = get_rebase_txs_OHM_staking_internal_txs(conn, cur)
    # get existing tx hashes that we have already processed (foreign key relationship to OHM_stake_internal_txs)
    txhashes = get_tx_hashes_OHM_rebase_txs(conn, cur)
    # filter internal txs to only get txs that have not yet been processed and added to OHM_rebase_txs
    txs_new = [tx for tx in txs if tx[1] not in txhashes]
    
# Parse logs for each tx in txs_new 
block_data = []
for block, txhash in txs_new: 
    res = parse_sohm_internal_tx_logs(txhash)
    if res: 
        res.block_no = block 
        res.date = dt.fromtimestamp(w3.eth.get_block(block).timestamp, tz=timezone.utc)
        block_data.append(res)
        print(f"epoch: {res.epoch} - rebase: {res.rebase} - index: {res.index} - totalSupply: {res.totalSupply}")

# Convert parsed tx logs representing rebase events into objects for db insertion 
block_data.sort(key=lambda e: e.epoch)
rebase_txs = []
for i in range(1, len(block_data)): 
    rebase_txs.append(
        AttrDict({
            "tx_hash": block_data[i].tx_hash, 
            "block_no": block_data[i].block_no, 
            "date": block_data[i].date, 
            "rebase": (block_data[i].index / block_data[i-1].index) - 1, 
            "epoch": block_data[i].epoch
        })
    )

with db_conn() as conn, db_cur(conn) as cur: 
    create_table_OHM_rebase_txs(conn, cur)
    insert_many_table_OHM_rebase_txs(conn, cur, rebase_txs)
    rebases = get_all_table_OHM_rebase_txs(conn, cur)

Skipping failed transaction
Skipping failed transaction
Skipping transaction that doesn't have logSupply and logRebase events
Skipping transaction that doesn't have logSupply and logRebase events
Skipping failed transaction
Skipping transaction that doesn't have logSupply and logRebase events
Skipping transaction that doesn't have logSupply and logRebase events
Skipping failed transaction
Skipping failed transaction
Skipping transaction that doesn't have logSupply and logRebase events
Skipping transaction that doesn't have logSupply and logRebase events
Skipping failed transaction
Skipping transaction that doesn't have logSupply and logRebase events
Skipping transaction that doesn't have logSupply and logRebase events
Skipping transaction that doesn't have logSupply and logRebase events
Skipping transaction that doesn't have logSupply and logRebase events
Skipping transaction that doesn't have logSupply and logRebase events
Skipping transaction that doesn't have logSupply and logRebase

In [13]:
addr = '0x93a98e245c71dbcace51b22904181f8779306576'
# addr = '0xe249d1bE97f4A716CDE0D7C5B6b682F491621C41' # mine 
staking_deploy_tx = w3.eth.get_transaction(OHM_STAKING_V2_DEPLOYMENT_TX)
staking_deploy_block = int(staking_deploy_tx["blockNumber"])
dmin = dt.fromtimestamp(int(eth.get_block_reward_by_block_number(staking_deploy_block)['timeStamp']), tz=timezone.utc)
dmax = dt.now(timezone.utc)
balances, tsmap = get_ohm_token_balance_for_address(addr, dmin, dmax, rebases, log=True)

2021-06-23 17:55:33+00:00 block: 12691856 epoch: 263 Rebase: (0.0001176): 0 -> 0.0
2021-06-24 01:42:45+00:00 block: 12694003 epoch: 264 Rebase: (0.0000585): 0.0 -> 0.0
2021-06-24 10:08:59+00:00 block: 12696223 epoch: 265 Rebase: (0.0000584): 0.0 -> 0.0
2021-06-24 18:22:43+00:00 block: 12698405 epoch: 266 Rebase: (0.0000576): 0.0 -> 0.0
2021-06-25 02:23:26+00:00 block: 12700613 epoch: 267 Rebase: (0.0000577): 0.0 -> 0.0
2021-06-25 10:49:01+00:00 block: 12702813 epoch: 268 Rebase: (0.0000571): 0.0 -> 0.0
2021-06-25 19:13:48+00:00 block: 12705021 epoch: 269 Rebase: (0.0000570): 0.0 -> 0.0
2021-06-26 03:23:37+00:00 block: 12707224 epoch: 270 Rebase: (0.0000561): 0.0 -> 0.0
2021-06-26 11:24:16+00:00 block: 12709404 epoch: 271 Rebase: (0.0000560): 0.0 -> 0.0
2021-06-26 19:52:25+00:00 block: 12711607 epoch: 272 Rebase: (0.0000555): 0.0 -> 0.0
2021-06-27 03:55:44+00:00 block: 12713809 epoch: 273 Rebase: (0.0000553): 0.0 -> 0.0
2021-06-27 12:18:15+00:00 block: 12716013 epoch: 274 Rebase: (0.000

In [14]:
block = 13783000
print(f"Balances for block {block} - manually validate at https://etherscan.io/tokencheck-tool")
print(f"Wallet: {addr}\n")
for symbol, ts_balance in balances.items(): 
    final_balance = ts_balance[-1].balance
    print(f"{symbol:5} final balance: {final_balance:f} - token contract address: {OHM_TOKEN_CONTRACT_ADDRS[symbol]}")    

Balances for block 13783000 - manually validate at https://etherscan.io/tokencheck-tool
Wallet: 0x93a98e245c71dbcace51b22904181f8779306576

OHM   final balance: 0.000000 - token contract address: 0x383518188c0c6d7730d91b2c03a03c837814a899
wsOHM final balance: 0.000000 - token contract address: 0xca76543cf381ebbb277be79574059e32108e3e65
gOHM  final balance: 5.674899 - token contract address: 0x0ab87046fBb341D058F17CBC4c1133F25a20a52f
sOHM  final balance: 0.000000 - token contract address: 0x04f2694c8fcee23e8fd0dfea1d4f5bb8c352111f


In [15]:
ds = dt(year=2021, month=6, day=1, tzinfo=timezone.utc)
de = dt.now(timezone.utc)
print("The date range over which we wish to visualize wallet portfolio value ")
print(f"Start: {ds}")
print(f"End:   {de}")

The date range over which we wish to visualize wallet portfolio value 
Start: 2021-06-01 00:00:00+00:00
End:   2021-12-17 00:35:30.906547+00:00


In [16]:
def get_ohm_token_prices(ds, de): 
    # Get prices for all OHM tokens from coingecko 
    sym_id = dict(ohm='olympus', gohm='governance-ohm', wsohm='wrapped-staked-olympus', sohm='staked-olympus')
    prices = dict()
    for sym in sym_id.keys(): 
        # TODO: Come up with better way to get sOHM price 
        # Currently, coingecko does not have accurate data for sOHM for some reason. Though sOHM trades at a slight 
        # premium to OHM as it is staked and collecting rebase rewards, OHM price is still a good proxy 
        cid = sym_id[sym] if sym != 'sohm' else sym_id['ohm']
        coin_prices = cg.get_coin_market_chart_range_by_id(cid, 'usd', ds, de)['prices']
        prices[sym] = [AttrDict(price=e[1], date=dt.fromtimestamp(e[0] / 1000, tz=timezone.utc)) for e in coin_prices] 
    return prices 
        
prices = get_ohm_token_prices(utc_timestamp(ds), utc_timestamp(de))

In [17]:
def pd_timestamp_remove_time(date_series): 
    return date_series.dt.date.apply(lambda date: pd.Timestamp(date))

def np_dt64_to_date(d): 
    return pd.Timestamp(d).to_pydatetime().date()

def print_df(df, nrows=None, head=True):
    if not nrows: 
        nrows = len(df)
    if head: 
        display(HTML(df.iloc[:nrows].to_html()))
    else: 
        display(HTML(df.iloc[-nrows:].to_html()))

def pd_dt_series_to_unix(series): 
    # https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#from-timestamps-to-epoch
    return (series.dt.tz_localize(None) - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')

def pd_dt_series_to_date(series): 
    return series.dt.tz_localize(None).dt.date.apply(lambda d: pd.Timestamp(d))

# Get all unique timestamps across all balance time series 
timestamps_all = []
for ts in balances.values(): 
    timestamps_all.extend([e.date for e in ts])
# Get all dates within the range over which we are showcasing portfolio holding value 
for pd_ts in pd.date_range(ds, de, freq='D'): 
    timestamps_all.append(pd_ts.to_pydatetime())
# Combine all dates within our range with the unique timestamps across all balance time series. This is our full time index 
timestamps_all = np.unique(timestamps_all)
assert len(set([type(e) for e in timestamps_all])) == 1
        
portfolio = list()
for symbol in balances.keys(): 

    # construct a data frame from our balance time series 
    df_bal = pd.DataFrame(balances[symbol]).rename(columns={'date': 'utc_datetime'})

    # construct dataframe with single column containing our full time index 
    df_dates = pd.DataFrame(timestamps_all, columns=['utc_datetime'])

    # Merge our balance time series into the full time index 
    df_bal = df_dates.merge(df_bal, how='left', on='utc_datetime').sort_values('utc_datetime')
    df_bal['date'] = pd_dt_series_to_date(df_bal['utc_datetime'])
    assert str(df_bal.utc_datetime.dtype) == 'datetime64[ns, UTC]'
    assert str(df_bal.date.dtype) == 'datetime64[ns]'

    # Forward fill balance information. Impute 0 to first row balance if there is no starting balance for earliest timestamp 
    if len(df_bal) and np.isnan(df_bal.iloc[0]['balance']): 
        df_bal.loc[[0], ['balance']] = 0
    df_bal['balance'] = df_bal['balance'].fillna(method='ffill')

    # Construct dataframe from price time series 
    price_ts = prices[symbol.lower()]
    df_price = pd.DataFrame(price_ts).rename(columns={'date': 'utc_datetime'})
    df_price['date'] = pd_dt_series_to_date(df_price['utc_datetime'])

    # Merge price data into the balance time series 
    df = df_bal.merge(df_price, on='date', how='outer', suffixes=('', '_')).drop(columns='utc_datetime_').sort_values('utc_datetime')
    assert len(df_bal) == len(df)
    
    # Some tokens (ex: gOHM) don't have as much price history as others. Since we won't have any holdings of these 
    # tokens on these days, it's safe to fill in these missing values with 0 
    df['price'] = df['price'].fillna(0) 

    # Compute portfolio holding value for the current token for each point in the full time index 
    df['balance_value'] = df.balance * df.price
    df['token'] = symbol 
    
    # Add df for this token to the portfolio
    portfolio.append(df)

df = pd.concat(portfolio)
df.head()
# print_df(df)

,utc_datetime,balance,date,price,balance_value,token
0,2021-06-01 00:00:00+00:00,0.0,2021-06-01,207.655752,0.0,OHM
1,2021-06-02 00:00:00+00:00,0.0,2021-06-02,200.317895,0.0,OHM
2,2021-06-03 00:00:00+00:00,0.0,2021-06-03,201.455071,0.0,OHM
3,2021-06-04 00:00:00+00:00,0.0,2021-06-04,196.210088,0.0,OHM
4,2021-06-05 00:00:00+00:00,0.0,2021-06-05,196.245913,0.0,OHM


In [45]:
import altair as alt 

df_final = df.loc[
    np.logical_and(
        df.date > pd.Timestamp(dt(year=2021, month=9, day=1)), 
        df.date < pd.Timestamp(dt(year=2021, month=12, day=14)), 
    )
] 
df_final['balance_value'] = df_final['balance_value'].round(2)
tokens = df_final.token.unique()

sym_color = {"OHM": "#E4572E", "gOHM": "#29335C", "sOHM": "#A8C686", "wsOHM": "#F3A712"}

# TODO: get sym_color from other notebook 
time_field = 'utc_datetime'
time_spec = 'utcyearmonthdatehoursminutes(utc_datetime):O'

# Create a selection that chooses the nearest point & selects based on x-value
nearest = alt.selection(type='single', nearest=True, on='mouseover', fields=[time_field], empty='none', clear='mouseout')

base = alt.Chart(df_final).encode(
    alt.X(time_spec, title="Timestamp"), 
).properties(
    title=f'OHM Token Holdings: {addr}',
    height=500, 
    width=1100,
)

# Stacked area chart 
area = base.mark_area(
    color=sym_color[symbol],
    interpolate='step-after',
).encode(
    alt.Y("sum(balance_value):Q", title="Portfolio Value"), 
    color="token:N"
)

# Vertical line + tooltip tracking portfolio value at given point in time 
rule = base.transform_pivot(
    'token', 
    value='balance_value', 
    groupby=[time_field]
).transform_calculate(
    total='+'.join([f'datum.{token}' for token in tokens])
).mark_rule(
    color='black'
).encode(
    opacity=alt.condition(nearest, alt.value(.6), alt.value(0)),
    tooltip=[
        alt.Tooltip(time_field, type='temporal', title="date"),
        *[alt.Tooltip(c, type='quantitative') for c in tokens], 
        alt.Tooltip('total', type='quantitative')
    ]
).add_selection(
    nearest
)

alt.layer(area, rule).configure_axis(
    titleFontSize=24
).configure_title(
    fontSize=24
)

alt.LayerChart(...)

In [19]:
# balances['wsOHM']
# balances['sOHM']
block = 13783000
print(f"Balances for block {block} - manually validate at https://etherscan.io/tokencheck-tool")
print(f"Wallet: {addr}\n")
for symbol, ts_balance in balances.items(): 
    final_balance = ts_balance[-1].balance
    print(f"{symbol:5} final balance: {final_balance:f} - token contract address: {OHM_TOKEN_CONTRACT_ADDRS[symbol]}")  

Balances for block 13783000 - manually validate at https://etherscan.io/tokencheck-tool
Wallet: 0x93a98e245c71dbcace51b22904181f8779306576

OHM   final balance: 0.000000 - token contract address: 0x383518188c0c6d7730d91b2c03a03c837814a899
wsOHM final balance: 0.000000 - token contract address: 0xca76543cf381ebbb277be79574059e32108e3e65
gOHM  final balance: 5.674899 - token contract address: 0x0ab87046fBb341D058F17CBC4c1133F25a20a52f
sOHM  final balance: 0.000000 - token contract address: 0x04f2694c8fcee23e8fd0dfea1d4f5bb8c352111f


In [21]:
# df_final[
#     np.logical_and(
#         np.logical_and(df_final.token == 'OHM', df_final.balance_value != 0), 
#         np.logical_and(df_final.token == 'sOHM', df_final.balance_value != 0)
#     )
# ]